# Implementation of the Quantum Gradient Algorithm

Given a d-dimensional function f:
\begin{equation}
f(\vec{x}) = y
\end{equation}
The Quantum Gradient Algorithm aims to estimate the gradient of this function at the origin. The problem statement of this problem includes oracular access to an operation $U_f$, defined as such:
\begin{equation}
U_f: \ket{\vec{x_1}}\ket{\vec{x_2}}\ket{0} \rightarrow \ket{\vec{x_1}}\ket{\vec{x_2}}\ket{\vec{f(x_1, x_2)}}
\end{equation}
Put in words, this function takes as input d registers of q qubits plus an output register, and applies the modular addition of the function evaluation of the input registers onto the output register.

## Defining the Oracle Matrix

To test the algorithm with a simple function, define f as:
\begin{equation}
f(x_1, x_2) = x_1 + x_2
\end{equation}
This function is linear and two-dimensional, and two registers of 4 qubits will be used as input.

In [7]:
import numpy as np
from quantumreg import QuantumRegister, QuantumGate
from util import binaryArray2Int, int2BinaryArray
import matplotlib.pyplot as plt

# d input registers of q qubits each
d = 2 
q = 3

# The function f is defined as such
def f(x1, x2):
    return x1 - x2

# Another function is defined to perform the bitwise action of applying the oracle to a set of input and output registers.
def applyMatrix(vec):
    x1 = binaryArray2Int(vec[0:0+q])
    x2 = binaryArray2Int(vec[q:2*q])
    y = binaryArray2Int(vec[2*q:3*q])
    new_y = (y + f(x1, x2)) % (2**q)
    output_binary = int2BinaryArray(new_y, q)
    
    for i, b in enumerate(output_binary):
        vec[2*q+i] = b
        
    return vec

In order to apply $U_f$ in this implementation, its matrix representation is needed. Just as with any operator on a quantum system, its matrix elements can be defined as follows:
\begin{equation}
(U_f)_{ij} = \bra{i}U_f\ket{j}
\end{equation}
The following routine passes the bit representation of every basis state and applies the behavior of the oracle to it. The resulting bit representation is then converted to the state vector representation. The inner product of this state vector with every other state vector is then found in order to define the row of matrix elements.

In [8]:
Uf_mat = [[0 for j in range(2**(3*q))] for k in range(2**(3*q))]

applymatrix_cache = {}

def int2EigenState(n: int):
    ret = np.array([[0] for i in range(2**(3*q))])
    ret[n] = 1
    return ret

for j in range(2**(3*q)):
    for k in range(2**(3*q)):
        bra = int2EigenState(j).transpose()
        ket = None
        if k not in applymatrix_cache:    
            rightbits = int2BinaryArray(k, 3*q)
            Ufrightbits = applyMatrix(rightbits)
            ket = int2EigenState(binaryArray2Int(Ufrightbits))
            applymatrix_cache[k] = ket
        else:
            ket = applymatrix_cache[k]
        
        Ufjk = np.dot(bra, ket)[0][0]
        Uf_mat[j][k] = Ufjk

## Defining Other Necessary Operations

The rest of this algorithm involves the use of hadamard gates and the quantum fourier transform. Both of these operations have known matrix elements that can be "hard-coded." These operations can be made to act only on certain bits by taking the tensor product of their matrix representations with the identity matrix.

In [9]:
# single qubit hadamard gate
hadamard = QuantumGate(np.divide(np.array([[1,1],[1,-1]]), np.sqrt(2)))

# single qubit identity
identity = QuantumGate(np.array([[1,0],[0,1]]))

# q-qubit quantum fourier transform
qft = QuantumGate([[0 for j in range(2**q)] for k in range(2**q)])
def w_n(n: int):
    return np.exp(2*np.pi*1.j / (2**n))
for j in range(2**q):
    for k in range(2**q):
        qft[j][k] = 1/np.sqrt(2**q) * w_n(q)**(j*k)

# q-qubit inverse fourier transform
qft_dag = QuantumGate(np.array(qft.mat).conj().transpose())

In [10]:
# step one: apply hadamard to input registers
one = (hadamard**(d*q)) * (identity**q)

# step two: apply qft to output registers
two = (identity**(d*q)) * (qft)

# step three: apply the oracle
three = QuantumGate(Uf_mat)

# step four: apply inverse qft to input registers
four = ((qft_dag**2) * (identity**q))

In [11]:
input = QuantumRegister(bits=[0]*(d*q) + [1]*q)

input = one*input
input = two*input
input = three*input
input = four*input

In [19]:
map = {}
form = "0" + str((3*q)) + "b"

for i in range(1000):
    result = format(input.measure(), form)
    if result not in map:
        map[result] = 1
    else:
        map[result] += 1
        
map

{'001111101': 115,
 '001111001': 134,
 '001111110': 130,
 '001111100': 114,
 '001111111': 119,
 '001111000': 121,
 '001111011': 134,
 '001111010': 133}